In [1]:
from model import RungeKuttaIntegratorCell, create_model, RNN, Sequential
import numpy as np
import pandas as pd

In [2]:
import os
import importlib

In [3]:
os.getcwd()

'C:\\Users\\p.attard\\OneDrive - C08759 - ENERGY POOL\\these\\biblio\\scientifi_machine_learning\\pinn_nascimento'

In [4]:
# masses, spring coefficients, and damping coefficients
alpha = np.array([0.2], dtype='float32')
T = 12.
omega = np.array([(2*np.pi / T)], dtype='float32')

In [5]:
df_all = pd.read_csv('./data.csv', sep=";", decimal=",", header=0)

In [6]:
df = df_all.loc[ df_all.loc[:, "case"] == 0, :]
df

,time,case,theta0,theta_dot0,u,tetha,theta_dot
0,0.0,0,0.192342,1.057883,0,0.138456,1.003997
1,0.5,0,0.192342,1.057883,0,0.631661,0.848859
2,1.0,0,0.192342,1.057883,0,1.015907,0.641543
3,1.5,0,0.192342,1.057883,0,1.375889,0.502063
4,2.0,0,0.192342,1.057883,0,1.658025,0.388829
...,...,...,...,...,...,...,...
76,38.0,0,0.192342,1.057883,0,0.021129,0.060014
77,38.5,0,0.192342,1.057883,0,-0.018472,0.005161
78,39.0,0,0.192342,1.057883,0,0.017835,0.026135
79,39.5,0,0.192342,1.057883,0,-0.008569,-0.014694


In [7]:
# data
t  = df.loc[:, "time"].values
dt = (t[1] - t[0])
utrain = df.loc[:, ["u"]].values[np.newaxis, :, :]
tetha_train = df.loc[:, ["tetha"]].values[np.newaxis, :, :]

In [8]:
utrain.shape

(1, 81, 1)

In [9]:
# fitting physics-informed neural network
initial_state = df.loc[:, ["theta0", "theta_dot0"]].iloc[[0], :].astype("float32").values
# create_model(omega, alpha, dt, initial_state, batch_input_shape, return_sequences=True, unroll=False)

In [10]:
model = create_model(omega, alpha, dt, initial_state=initial_state, batch_input_shape=utrain.shape)

===inputs===
Tensor("rnn/strided_slice_2:0", shape=(1, 1), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
(1, 1)
===alpha===
<tf.Variable 'rnn/runge_kutta_integrator_cell/ALPHA:0' shape=(1,) dtype=float32>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
(1,)
===omega===
[0.5235988]
<class 'numpy.ndarray'>
(1,)
===inputs===
Tensor("rnn/strided_slice_2:0", shape=(1, 1), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
(1, 1)
===alpha===
<tf.Variable 'rnn/runge_kutta_integrator_cell/ALPHA:0' shape=(1,) dtype=float32>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
(1,)
===omega===
[0.5235988]
<class 'numpy.ndarray'>
(1,)
===inputs===
Tensor("rnn/strided_slice_2:0", shape=(1, 1), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
(1, 1)
===alpha===
<tf.Variable 'rnn/runge_kutta_integrator_cell/ALPHA:0' shape=(1,) dtype=float32>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
(1

NotImplementedError: in user code:

    C:\Users\p.attard\OneDrive - C08759 - ENERGY POOL\these\biblio\scientifi_machine_learning\pinn_nascimento\model.py:39 call  *
        theta_ddoti = self._fun(self._omega, alpha, inputs, theta, theta_dot)
    C:\Users\p.attard\OneDrive - C08759 - ENERGY POOL\these\biblio\scientifi_machine_learning\pinn_nascimento\model.py:75 _fun  *
        return inputs - (omega ** 2) * np.sin(theta) - alpha * theta_dot
    c:\users\p.attard\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\ops.py:867 __array__  **
        raise NotImplementedError(

    NotImplementedError: Cannot convert a symbolic Tensor (rnn/while/runge_kutta_integrator_cell/strided_slice_1:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported


In [ ]:
rkCell = RungeKuttaIntegratorCell(omega=omega, alpha=alpha, dt=dt, initial_state=initial_state)

In [ ]:
PINN = RNN(cell=rkCell, batch_input_shape=utrain.shape, return_sequences=True, return_state=False,
               unroll=False)

In [12]:
model = Sequential()

In [13]:
model.add(PINN)

NotImplementedError: in user code:

    C:\Users\p.attard\OneDrive - C08759 - ENERGY POOL\these\biblio\scientifi_machine_learning\pinn_nascimento\model.py:39 call  *
        theta_ddoti = self._fun(self._omega, alpha, inputs, theta, theta_dot)
    C:\Users\p.attard\OneDrive - C08759 - ENERGY POOL\these\biblio\scientifi_machine_learning\pinn_nascimento\model.py:63 _fun  *
        return inputs - (omega ** 2) * np.sin(theta) - alpha * theta_dot
    c:\users\p.attard\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\ops.py:867 __array__  **
        raise NotImplementedError(

    NotImplementedError: Cannot convert a symbolic Tensor (rnn/while/runge_kutta_integrator_cell/strided_slice_1:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported


In [ ]:
model.compile(loss='mse', optimizer=RMSprop(1e4), metrics=['mae'])

In [ ]:
yPred_before = model.predict_on_batch(utrain)[0, :, :]
model.fit(utrain, ytrain, epochs=100, steps_per_epoch=1, verbose=1)
yPred = model.predict_on_batch(utrain)[0, :, :]

In [20]:
# fitting physics-informed neural network
initial_state = np.zeros((1,2 * len(m),), dtype='float32')
model = create_model(m, c, k, dt, initial_state=initial_state, batch_input_shape=utrain.shape)
yPred_before = model.predict_on_batch(utrain)[0, :, :]
model.fit(utrain, ytrain, epochs=100, steps_per_epoch=1, verbose=1)
yPred = model.predict_on_batch(utrain)[0, :, :]

NameError: name 'm' is not defined